In [1]:
# Initialize the environment
using Pkg
Pkg.activate(".")
Pkg.instantiate()
ENV["DATADEPS_ALWAYS_ACCEPT"] = true # accept the download of the MNIST dataset
SHOW_PROGRESS_BAR = true # turn off progress bars

  Activating project at `c:\Users\mkornjaca\Documents\GitHub\BloqadeReservoir.jl\notebooks`


true

# Quantum Reservoir Computing Demo

In this notebook we will show you how to train a model to classify MNIST images using quantum reservoir computing (QRC) in Julia programming language. The general idea is that the chaotic, non-linear dynamics of quantum systems can be used to create a non-linear mapping from a low-dimensional space (which will be PCA embeddings extracted from the MNIST images) to a high dimensional space. In order to construct the mapping, we first ensure that information from the image will totally specify the dynamics of the quantum system. Then, we measure certain properties (in this case, spin expectations and correlations) over time. These measurements form the output of the mapping.   

Through the embedding in this high dimensional space classification on using standard ML techniques is expected to perform better. 

This demo provides a toy model that does not require access to a quantum computer. It uses classical, numerical simulations of a quantum system with 10 qubits. The classical simulation makes use of the package Bloqade, which is designed for numerical simulations of neutral-atom architectures. 

Neutral atom architectures provide access to several variational parameters. In this demo, we map pixelated descriptions of the images to the local detuning terms in the Rydberg Hamiltonian. We then measure the quantum system and train the classical model which maps the quantum readout from the measurement to the desired prediction. As all training process is in the classical part (excuted on a classical computer), we expect the training is substantially shorter than other traditional quantum machine learning models. More details on numerical simulation of neutral-atom architectures, and the range of the parameters can be found from [Bloqade.jl](https://queracomputing.github.io/Bloqade.jl/dev/) documentation. 

In [2]:
# import required libraries
using MLDatasets
using MultivariateStats
using OneHotArrays
using Flux
using Bloqade
using Colors
using ProgressBars
using JLD2

In [3]:
# Download the MNIST dataset

# training data set (each image has 28x28 pixels and there are 60000 training samples)
data_train = MNIST(:train)
# test data set (10000 test samples)
data_test = MNIST(:test) 

dataset MNIST:
  metadata  =>    Dict{String, Any} with 3 entries
  split     =>    :test
  features  =>    28×28×10000 Array{Float32, 3}
  targets   =>    10000-element Vector{Int64}

In [4]:
# example of an training dataset image for '5'
Gray.(data_train.features[:, :, 1]')

## PCA Reduction

In this notebook, we first train a toy model using 1000 samples from the MNIST dataset.

The images form the input into the quantum system. As we focus on numerical simulation where the number of atoms in the quantum system is limited, we first perform dimensionality reduction using the [principal component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis). 

In [5]:
# We first use PCA to downsample the data into 10-dimensional vectors
dim_pca = 10

# Use the `fit` function from the `MultivariateStats` package to generate the projection operator for PCA
model_pca = fit(PCA, reshape(data_train.features, (784, 60000)); maxoutdim=dim_pca);

# Use the `predict` function from `MultivariateStats` package to compute the first 10 principal components
x = predict(model_pca, reshape(data_train.features, (784, 60000)))

# Let us see how it looks
num_examples = 1000
xs = x[:, 1:num_examples]

10×1000 Matrix{Float32}:
  0.485964    3.96748   -0.203377    …   2.05645     1.0019     -1.20236
 -1.22618    -1.1563     1.53793         0.0576729  -2.91969    -0.586595
  0.0961505  -2.33857    0.739275       -4.55099     1.7512     -2.84139
 -2.1794     -1.80688    2.04322        -1.15914     0.61376     1.58327
 -0.10712    -3.24429   -1.20275        -2.25572    -1.38172     1.01346
 -0.911679   -0.713541  -0.00720531  …   0.168004   -0.135651   -0.982429
 -0.91759     0.176592   3.36885         1.44196    -0.0641531   0.427178
  0.626676   -0.411636   1.44547         2.03504     2.08525     2.04952
  1.42557    -0.158639   0.449218       -0.594718    0.705034   -1.49505
 -0.778144   -0.592028   0.700038        0.674973    0.445749   -0.120354

Here, we scale the range of principal components to a feasible range of local detuning (we use $[-6\ \text{rad}/\mu s,6\ \text{rad}/\mu s]$ for the toy model). And later, for each image, we will encode each of the 10 scaled principal components into each single local detuning for 10 atoms. For more details of the neutral atom quantum system, please refer to the documentation of [Bloqade.jl](https://queracomputing.github.io/Bloqade.jl/dev/#What-does-Bloqade-Do?).

In [6]:
Δ_max = 6.0
spectral = max(abs(maximum(xs)), abs(minimum(xs)))
xs = xs/spectral * Δ_max # to make sure values to be between [-6.0, 6.0]

10×1000 Matrix{Float64}:
  0.380702    3.10811   -0.159325    …   1.61102     0.784881   -0.941924
 -0.960581   -0.905843   1.20481         0.0451807  -2.28728    -0.459536
  0.0753239  -1.83203    0.579145       -3.56522     1.37188    -2.22593
 -1.70733    -1.4155     1.60065        -0.908064    0.480817    1.24033
 -0.0839175  -2.54156   -0.942226       -1.76712    -1.08243     0.793937
 -0.714206   -0.558985  -0.00564461  …   0.131614   -0.106269   -0.769631
 -0.718836    0.138341   2.63915         1.12962    -0.0502573   0.33465
  0.490935   -0.322474   1.13237         1.59424     1.63358     1.60558
  1.11679    -0.124277   0.351916       -0.4659      0.552321   -1.17122
 -0.609595   -0.463793   0.548407        0.528771    0.349198   -0.0942845

We are using [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) for the labels from 0 to 9.

In [7]:
y = onehotbatch(data_train.targets, 0:9)
# Let us see how labels look
ys = y[:, 1:num_examples]

10×1000 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  1     ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     1  ⋅  1  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  1
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

## Simulate Quantum Dynamics

We can now set up the quantum simulation. 

- Define a `struct`, which captures all physical parameters and readouts of the quantum system.
- Define functions that simulate quantum dynamics with the input `x` from scaled detunings and return the output `out` of readout observables
- The simulation of the quantum dynamics has been implemented in [Bloqade.jl](https://queracomputing.github.io/Bloqade.jl/dev/)

In [8]:
# Define the struct
Base.@kwdef struct DetuningLayer
    atoms # atom positions
    readouts # readout observables
    Ω::Float64 # Rabi frequency
    t_start::Float64 # evolution starting time
    t_end::Float64 # evolution ending time
    step::Float64  # readout time step
    reg::AbstractArrayReg # quantum state storage
end

DetuningLayer

In [9]:
# Iterate over each single image to simulate the quantum evolution and record the output
# For more details on simulation, please refer to Bloqade.jl and Yao.jl
function apply_layer(layer::DetuningLayer, x::Vector{<:Real})
    # define Rydberg hamiltonian, detunings parameterized in terms of pca values (x)  
    h = rydberg_h(layer.atoms; Δ = x, Ω = layer.Ω) 
    
    # At the start of the simulation, all atoms of the system are in ground states
    reg = layer.reg
    set_zero_state!(reg) 
    
    t_start = layer.t_start
    t_end = layer.t_end
    t_step = layer.step
    
    # initialize output vector
    steps = floor(Int, (t_end - t_start) / t_step)
    out = zeros(steps * length(layer.readouts))
    
    # Numerically simulate the quantum evolution with Krylov methods and store the readouts
    i = 1
    prob =  KrylovEvolution(reg, layer.t_start:layer.step:layer.t_end, h)
    for (step, reg, _) in prob # step through the state at each time step 
        step == 1 && continue # ignore first time step, this is just the initial state
        for op in layer.readouts
            out[i] = real(expect(op, reg)) # store the expectation of each operator for the given state in the output vector 
            i+=1
        end
    end
    return out
end

# implement functions that apply a `DetuningLayer` to a matrix containing scaled detunings for each image
function apply_layer(layer::DetuningLayer, x::Matrix{<:Real})
    iter = SHOW_PROGRESS_BAR ? ProgressBar(1:size(x, 2)) : 1:size(x, 2)
    outs = [apply_layer(layer, x[:, i][:]) for i in iter]
    return hcat(outs...)
end

apply_layer (generic function with 2 methods)

Now we use the above struct to describe our toy model. We prepare the 10 atoms in a chain lattice and use a numerical simulation of the quantum dynamics associated to the input.

In [10]:
# Generate atom positions for the toy model
d = 10.0
atoms = generate_sites(ChainLattice(), dim_pca; scale = d); # put atoms in a chain with 10 micron spacing

# create all single site Zᵢ and correlator ZᵢZⱼ readouts 
nsites = length(atoms)
readouts = AbstractBlock[put(nsites, i => Z) for i in 1:nsites]
for i in 1:nsites
    for j in i+1:nsites
        push!(readouts, chain(put(nsites, i => Z), put(nsites, j => Z)))
    end
end

# build preprocessing layer 
pre_layer = DetuningLayer(;
    atoms, 
    readouts, 
    Ω = 2π, 
    t_start = 0.0, 
    t_end = 4.0, 
    step = 0.5, 
    reg = zero_state(nsites)
);

Now, we are ready to run the simulation by apply the defined `pre_layer` to the scaled detunings! For each image, the readouts from the quantum dynamics is a 440-dimensional vector which has much higher dimension than the PCA dimension. The full results will be stored in a $440\times 1000$ matrix `embeddings`.

(It might take a few minutes)

In [11]:
# uncomment the next line to see progress bar
SHOW_PROGRESS_BAR = true 
embeddings = apply_layer(pre_layer, xs)

0.0%┣                                             ┫ 0/1.0k [00:00<00:-3, -0s/it]


0.1%┣                                         ┫ 1/1.0k [00:10<Inf:Inf, InfGs/it]
0.3%┣▏                                          ┫ 3/1.0k [00:10<01:25:05, 5s/it]


0.5%┣▎                                             ┫ 5/1.0k [00:10<42:47, 3s/it]
0.7%┣▎                                             ┫ 7/1.0k [00:10<28:40, 2s/it]


0.9%┣▍                                             ┫ 9/1.0k [00:10<21:36, 1s/it]
1.1%┣▌                                            ┫ 11/1.0k [00:11<17:21, 1s/it]


1.2%┣▌                                            ┫ 12/1.0k [00:11<15:53, 1it/s]
1.4%┣▋                                            ┫ 14/1.0k [00:11<13:31, 1it/s]


...

98.5%┣█████████████████████████████████████████▍┫ 985/1.0k [00:41<00:01, 24it/s]
98.7%┣█████████████████████████████████████████▌┫ 987/1.0k [00:41<00:01, 24it/s]


98.9%┣█████████████████████████████████████████▌┫ 989/1.0k [00:41<00:00, 24it/s]
99.1%┣█████████████████████████████████████████▋┫ 991/1.0k [00:41<00:00, 24it/s]


99.3%┣█████████████████████████████████████████▊┫ 993/1.0k [00:42<00:00, 24it/s]
99.5%┣█████████████████████████████████████████▉┫ 995/1.0k [00:42<00:00, 24it/s]


99.7%┣█████████████████████████████████████████▉┫ 997/1.0k [00:42<00:00, 24it/s]
99.9%┣██████████████████████████████████████████┫ 999/1.0k [00:42<00:00, 24it/s]
100.0%┣████████████████████████████████████████┫ 1.0k/1.0k [00:42<00:00, 24it/s]
100.0%┣████████████████████████████████████████┫ 1.0k/1.0k [00:42<00:00, 24it/s]


440×1000 Matrix{Float64}:
 -0.630605   -0.749976     -0.49692     …  -0.713219   -0.377042
  0.207054    0.121035     -0.173302        0.411674    0.0479746
 -0.0909969   0.285497     -0.0749144      -0.301975    0.389858
  0.304526    0.145654     -0.295578       -0.09762    -0.271875
 -0.0511498   0.415413      0.18172         0.19774    -0.146604
  0.111823    0.0328114    -0.00654347  …  -0.0140813   0.150889
  0.119965   -0.0458108    -0.416157        0.022207   -0.0634113
 -0.0940252   0.0534113    -0.181878       -0.281912   -0.306508
 -0.163187    0.0455845    -0.0148809      -0.0473649   0.249001
 -0.411159   -0.466793     -0.626231       -0.593586   -0.565491
  ⋮                                     ⋱              
 -0.0217969   0.0641197    -0.0774998       0.0122162   0.106881
 -0.0635694  -0.113028     -0.198293       -0.100126   -0.14746
 -0.0255301   0.0936819    -0.0945152      -0.0771544  -0.00242692
 -0.11437    -0.122263      0.147018       -0.185221   -0.0863102
  0.

## Train Neural Network

Now we are going to train a classical neural network without hidden layers (a linear classifier with `softmax` output function), using quantum measurements stored in `embeddings` as the input. In comparision, we also trained a neural network without hidden layers directly using PCA feature vectors without the quantum reservoir processing.

Here, we used the machine learning framework [Flux.jl](https://fluxml.ai/Flux.jl/stable/) in Julia for the neural network training.

First, we define the function `train` for training.

In [12]:
function train(xs, ys)
    # Build a simple neural network model
    model_atoms = Chain(
        Dense(length(xs[:, 1]), 10),
        softmax
    )

    # Define the training data loader, trainable model parameters, and the optimizer
    loader = Flux.DataLoader((xs, ys), batchsize=100, shuffle=true);
    ps = Flux.params(model_atoms)
    opt = Flux.Adam(0.01)

    # Train the model. We use the crossentropy as a loss function
    nepochs = 1000
    losses = []
    train_iter = SHOW_PROGRESS_BAR ? ProgressBar(1:nepochs) : 1:nepochs
    for epoch in train_iter
        l = 1.0
        for (x, y) in loader
            grads = Flux.gradient(ps) do
                ŷ = model_atoms(x)
                l = Flux.crossentropy(ŷ, y)
            end
            Flux.update!(opt, ps, grads)
        end
        push!(losses, Flux.crossentropy(model_atoms(xs), ys))  # logging, outside gradient context
    end
    return model_atoms, losses
end

train (generic function with 1 method)

Call the above function to train the classical model directly using the principal components without the quantum reservoir processing.

In [13]:
model_reg, loss_reg = train(xs, ys)

0.0%┣                                             ┫ 0/1.0k [00:00<00:00, -0s/it]


┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(10 => 10)
│   summary(x) = 10×100 Matrix{Float64}
└ @ Flux C:\Users\mkornjaca\.julia\packages\Flux\EHgZm\src\layers\stateless.jl:60


0.1%┣                                         ┫ 1/1.0k [00:09<Inf:Inf, InfGs/it]
5.5%┣██▌                                          ┫ 55/1.0k [00:09<02:39, 6it/s]


12.7%┣█████▍                                    ┫ 127/1.0k [00:09<01:03, 14it/s]
20.9%┣████████▉                                 ┫ 209/1.0k [00:09<00:35, 23it/s]


29.4%┣████████████▍                             ┫ 294/1.0k [00:09<00:22, 32it/s]
38.8%┣████████████████▎                         ┫ 388/1.0k [00:09<00:15, 42it/s]


47.7%┣████████████████████                      ┫ 477/1.0k [00:09<00:10, 51it/s]
56.1%┣███████████████████████▋                  ┫ 561/1.0k [00:09<00:07, 60it/s]


64.9%┣███████████████████████████▎              ┫ 649/1.0k [00:09<00:05, 69it/s]
73.6%┣███████████████████████████████           ┫ 736/1.0k [00:09<00:03, 78it/s]


83.2%┣███████████████████████████████████       ┫ 832/1.0k [00:10<00:02, 87it/s]
91.0%┣██████████████████████████████████████▏   ┫ 910/1.0k [00:10<00:01, 95it/s]


98.5%┣████████████████████████████████████████▍┫ 985/1.0k [00:10<00:00, 102it/s]
100.0%┣███████████████████████████████████████┫ 1.0k/1.0k [00:10<00:00, 104it/s]
100.0%┣███████████████████████████████████████┫ 1.0k/1.0k [00:10<00:00, 104it/s]


(Chain(Dense(10 => 10), softmax), Any[2.536362f0, 2.0674543f0, 1.7146058f0, 1.4598562f0, 1.2792201f0, 1.150938f0, 1.0577667f0, 0.9892145f0, 0.9369046f0, 0.894655f0  …  0.5622521f0, 0.5622054f0, 0.56217635f0, 0.56220645f0, 0.56218016f0, 0.56229997f0, 0.5622145f0, 0.5622725f0, 0.5622063f0, 0.5623234f0])

Call the above function to train the neural network using the readouts from the quantum reservoir processing as the input.

In [14]:
model_qrc, loss_qrc = train(embeddings, ys)

0.0%┣                                             ┫ 0/1.0k [00:00<00:00, -0s/it]
0.6%┣▎                                            ┫ 6/1.0k [00:00<00:11, 94it/s]


1.3%┣▋                                          ┫ 13/1.0k [00:00<00:09, 115it/s]
2.0%┣▉                                          ┫ 20/1.0k [00:00<00:08, 122it/s]


2.8%┣█▏                                         ┫ 28/1.0k [00:00<00:08, 129it/s]
3.7%┣█▋                                         ┫ 37/1.0k [00:00<00:07, 135it/s]


4.4%┣██                                         ┫ 44/1.0k [00:00<00:07, 135it/s]
5.2%┣██▎                                        ┫ 52/1.0k [00:00<00:07, 136it/s]


6.0%┣██▋                                        ┫ 60/1.0k [00:00<00:07, 138it/s]
6.8%┣███                                        ┫ 68/1.0k [00:00<00:07, 139it/s]


...

93.8%┣██████████████████████████████████████▌  ┫ 938/1.0k [00:06<00:00, 156it/s]
94.7%┣██████████████████████████████████████▉  ┫ 947/1.0k [00:06<00:00, 156it/s]


95.5%┣███████████████████████████████████████▏ ┫ 955/1.0k [00:06<00:00, 156it/s]
96.3%┣███████████████████████████████████████▌ ┫ 963/1.0k [00:06<00:00, 156it/s]


97.1%┣███████████████████████████████████████▉ ┫ 971/1.0k [00:06<00:00, 156it/s]
98.0%┣████████████████████████████████████████▏┫ 980/1.0k [00:06<00:00, 156it/s]


98.9%┣████████████████████████████████████████▌┫ 989/1.0k [00:06<00:00, 156it/s]
99.7%┣████████████████████████████████████████▉┫ 997/1.0k [00:06<00:00, 156it/s]
100.0%┣███████████████████████████████████████┫ 1.0k/1.0k [00:06<00:00, 156it/s]
100.0%┣███████████████████████████████████████┫ 1.0k/1.0k [00:06<00:00, 156it/s]


(Chain(Dense(440 => 10), softmax), Any[1.2511722f0, 0.85587174f0, 0.70040244f0, 0.62526f0, 0.5623264f0, 0.5268328f0, 0.50204515f0, 0.47448325f0, 0.45120063f0, 0.43249232f0  …  0.00040387173f0, 0.00040052764f0, 0.0003997118f0, 0.00039794075f0, 0.0003959399f0, 0.00039481506f0, 0.0003951898f0, 0.00039274056f0, 0.00038856568f0, 0.00038883116f0])

## Model evaluation and prediction
Once we got the trained model, we can evaluate the prediction accuracy on the test set. 

We use 100 samples from the test set for evaluation.

In [15]:
num_test_examples = 100

# using the same PCA model we fit to the test set
test_features = predict(model_pca, reshape(data_test.features, (784, 10000)))[:, 1:num_test_examples] 

# same rescaling we applied on train input get reasonable detuning values 
test_features_qrc = test_features/spectral * Δ_max 

# quantum embeddings for 100 test samples
test_embeddings = apply_layer(pre_layer, test_features_qrc)

0.0%┣                                              ┫ 0/100 [00:00<00:00, -0s/it]


2.0%┣█                                             ┫ 2/100 [00:00<00:05, 18it/s]
4.0%┣█▉                                            ┫ 4/100 [00:00<00:04, 23it/s]


6.0%┣██▊                                           ┫ 6/100 [00:00<00:05, 21it/s]
7.0%┣███▏                                          ┫ 7/100 [00:00<00:05, 20it/s]


9.0%┣████▏                                         ┫ 9/100 [00:00<00:04, 22it/s]
11.0%┣████▉                                       ┫ 11/100 [00:00<00:04, 24it/s]


13.0%┣█████▊                                      ┫ 13/100 [00:00<00:03, 25it/s]
15.0%┣██████▋                                     ┫ 15/100 [00:01<00:03, 26it/s]


...

88.0%┣██████████████████████████████████████▊     ┫ 88/100 [00:03<00:00, 29it/s]


89.0%┣███████████████████████████████████████▏    ┫ 89/100 [00:03<00:00, 28it/s]
91.0%┣████████████████████████████████████████    ┫ 91/100 [00:03<00:00, 28it/s]


93.0%┣█████████████████████████████████████████   ┫ 93/100 [00:03<00:00, 28it/s]
95.0%┣█████████████████████████████████████████▉  ┫ 95/100 [00:03<00:00, 28it/s]


97.0%┣██████████████████████████████████████████▊ ┫ 97/100 [00:03<00:00, 28it/s]
99.0%┣███████████████████████████████████████████▋┫ 99/100 [00:03<00:00, 28it/s]
100.0%┣██████████████████████████████████████████┫ 100/100 [00:03<00:00, 28it/s]
100.0%┣██████████████████████████████████████████┫ 100/100 [00:03<00:00, 28it/s]


440×100 Matrix{Float64}:
 -0.311869    -0.65064      0.0413229   …  -0.699645    -0.612809
 -0.361863     0.505179     0.214676        0.0969915   -0.381465
 -0.0279459   -0.162273     0.0174723       0.0492573   -0.29476
  0.136522    -0.212432    -0.196317       -0.325904    -0.187485
  0.264492     0.00694864   0.30254         0.0622595   -0.259552
 -0.210657     0.238788    -0.097201    …   0.255166    -0.154559
  0.236685     0.131744    -0.0186813      -0.0423999   -0.0363814
 -0.27158     -0.164337     0.0500943       0.222611    -0.0418998
 -0.164413     0.124531     0.0831739      -0.225847     0.197264
 -0.617464    -0.601893    -0.619638       -0.546056    -0.305257
  ⋮                                     ⋱               
  0.0948437    0.0720948    0.183566        0.00378315   0.120916
 -0.125283    -0.145219    -0.039836       -0.214709    -0.104621
 -0.0423955    0.026556    -0.00545409     -0.128135     0.104058
 -0.292517    -0.203089    -0.126803       -0.174202     0.

We define a function to calculate the accuracy by camparing the predicting result and the label.

In [16]:
accuracy(model, xs, targets) = sum(onecold(model(xs), 0:9) .== targets)/length(targets)

accuracy (generic function with 1 method)

Let us see the performance of the classical model and the QRC model.

In [17]:
# correct labels for 100 test samples
test_targets = data_test.targets[1:num_test_examples]

# accuracy on training and test for each model
acc_train_reg = accuracy(model_reg, xs, onecold(ys, 0:9))
acc_test_reg = accuracy(model_reg,  test_features, test_targets)
acc_train_qrc = accuracy(model_qrc, embeddings, onecold(ys, 0:9))
acc_test_qrc = accuracy(model_qrc, test_embeddings, test_targets)

println("train accuracy of model on PCA embeddings: ", acc_train_reg)
println("test accuracy of model on PCA embeddings: ", acc_test_reg)
println("train accuracy of model on QRC embeddings: ", acc_train_qrc)
println("test accuracy of model on QRC embeddings: ", acc_test_qrc)

train accuracy of model on PCA embeddings: 0.826
test accuracy of model on PCA embeddings: 0.74
train accuracy of model on QRC embeddings: 1.0
test accuracy of model on QRC embeddings: 0.81


Because the number of training samples is not large enough, the QRC model got overfitted, so it may perform less well on prediction. To get better performance, we need more samples in the training set and more qubits will also help.

## Wrapping QRC pipeline
In the following part, we will wrap all the pipeline of the QRC model into a `struct` that takes images as the input and outputs the predicted labels.

In [18]:
# define the data structure for wrapping up
struct QRCModel
    model_pca
    spectral
    Δ_max
    detuning_layer
    linear_regression
end

# given input images compute the prediction
function (model::QRCModel)(xs)
    if length(size(xs)) == 3 # list of images
        num_samples = size(xs)[end]
        num_sample_dim = prod(size(xs)[1:end-1])
    elseif length(size(xs)) == 2 # one image
        num_samples = 1
        num_sample_dim = prod(size(xs))
    end
    xs = reshape(xs, (num_sample_dim, num_samples)) # flatten the input
    xs_pca = predict(model.model_pca, xs)
    xs_pca = xs_pca / model.spectral * model.Δ_max
    detuning_layer = model.detuning_layer
    num_steps = length(detuning_layer.t_start:detuning_layer.step:detuning_layer.t_end) - 1
    embeddings = apply_layer(detuning_layer, xs_pca)
    ys = model.linear_regression(embeddings)
    predicts = onecold(ys, 0:9)
    return predicts
end

Once the pipeline of the QRC model has been wrapped, we will compare the toy model we have just built with a pre-trained model. The pre-trained model has been trained on the MNIST dataset using 10000 samples and 15 atoms, and has been saved in the `pre_trained.jld` file.

In [19]:
# This is the toy model we trained from 1000 images
toy_model = QRCModel(model_pca, spectral, Δ_max, pre_layer, model_qrc);

# This is a pre-trained model we trained from 10000 images with 15 atoms
pre_trained_model = load("pre_trained.jld2")["qrc_model"];

We select the first 100 images from the test set.

In [20]:
num_test_examples = 100
test_examples = data_test.features[:,:,1:num_test_examples]
label = data_test.targets[1:num_test_examples];

Let us see the performance of the toy model on these 100 images

In [21]:
pred_toy = toy_model(test_examples)
sum(pred_toy .== label) / length(label)

0.0%┣                                              ┫ 0/100 [00:00<00:00, -0s/it]


1.0%┣▍                                         ┫ 1/100 [00:00<Inf:Inf, InfGs/it]
3.0%┣█▍                                            ┫ 3/100 [00:00<00:06, 16it/s]


5.0%┣██▎                                           ┫ 5/100 [00:00<00:05, 21it/s]
7.0%┣███▏                                          ┫ 7/100 [00:00<00:04, 24it/s]


9.0%┣████▏                                         ┫ 9/100 [00:00<00:03, 26it/s]
11.0%┣████▉                                       ┫ 11/100 [00:00<00:03, 27it/s]


13.0%┣█████▊                                      ┫ 13/100 [00:00<00:03, 28it/s]
15.0%┣██████▋                                     ┫ 15/100 [00:00<00:03, 28it/s]


...

85.0%┣█████████████████████████████████████▍      ┫ 85/100 [00:03<00:00, 33it/s]
87.0%┣██████████████████████████████████████▎     ┫ 87/100 [00:03<00:00, 33it/s]


89.0%┣███████████████████████████████████████▏    ┫ 89/100 [00:03<00:00, 33it/s]
91.0%┣████████████████████████████████████████    ┫ 91/100 [00:03<00:00, 33it/s]


93.0%┣█████████████████████████████████████████   ┫ 93/100 [00:03<00:00, 33it/s]
95.0%┣█████████████████████████████████████████▉  ┫ 95/100 [00:03<00:00, 33it/s]


97.0%┣██████████████████████████████████████████▊ ┫ 97/100 [00:03<00:00, 33it/s]
99.0%┣███████████████████████████████████████████▋┫ 99/100 [00:03<00:00, 33it/s]
100.0%┣██████████████████████████████████████████┫ 100/100 [00:03<00:00, 33it/s]
100.0%┣██████████████████████████████████████████┫ 100/100 [00:03<00:00, 33it/s]


0.81

Let us see the performance of the pre-trained model on these 100 images.

(It may take a few minutes to run the dynamics)

In [22]:
# uncomment the next line to see progress bar
# SHOW_PROGRESS_BAR = true 
pred_pre_trained = pre_trained_model(test_examples)
sum(pred_pre_trained .== label) / length(label)

0.0%┣                                              ┫ 0/100 [00:00<00:00, -0s/it]


1.0%┣▍                                         ┫ 1/100 [00:02<Inf:Inf, InfGs/it]


2.0%┣█                                              ┫ 2/100 [00:03<05:11, 3s/it]


3.0%┣█▍                                             ┫ 3/100 [00:05<03:46, 2s/it]


4.0%┣█▉                                             ┫ 4/100 [00:06<03:18, 2s/it]


...

97.0%┣███████████████████████████████████████████▋ ┫ 97/100 [02:29<00:05, 2s/it]


98.0%┣████████████████████████████████████████████ ┫ 98/100 [02:31<00:03, 2s/it]


99.0%┣████████████████████████████████████████████▌┫ 99/100 [02:32<00:02, 2s/it]


100.0%┣███████████████████████████████████████████┫ 100/100 [02:34<00:00, 2s/it]
100.0%┣███████████████████████████████████████████┫ 100/100 [02:34<00:00, 2s/it]


0.97

We randomly select a sample from the test set and make the prediction with our toy model and the pre-trained model.

In [38]:
sample_id = rand(1:10000)
random_sample = data_test.features[:,:, sample_id]
Gray.(random_sample')

Now we use these two models for prediction.

In [39]:
toy_model(random_sample)

0.0%┣                                                ┫ 0/1 [00:00<00:00, -0s/it]
100.0%┣██████████████████████████████████████████┫ 1/1 [00:00<Inf:Inf, InfGs/it]
100.0%┣██████████████████████████████████████████┫ 1/1 [00:00<Inf:Inf, InfGs/it]


1-element Vector{Int64}:
 3

In [40]:
pre_trained_model(random_sample)

0.0%┣                                                ┫ 0/1 [00:00<00:00, -0s/it]


100.0%┣██████████████████████████████████████████┫ 1/1 [00:01<Inf:Inf, InfGs/it]
100.0%┣██████████████████████████████████████████┫ 1/1 [00:01<Inf:Inf, InfGs/it]


1-element Vector{Int64}:
 3